#  Using a Custom Tensorflow Object Detection Model with this PanoramSDK Emulator

**Goal of this Notebook** :

* Showcase how to use a custom build Tensorflow Object Detection model with this Emulator
* Using the built in wrapper application that **mimics** the Panorama sdk to get inference from the model

**What this Notebook accomplishes?** :
* Detect People in a selected video
* Draw bounding boxes around the people
* Count the number of people detected
* Display the count on the video frames


**Useful Resources to aid your development**:
* [AWS Panorama Documentation](https://docs.aws.amazon.com/panorama/)



**CAUTION PLEASE READ BEFORE PROCEEDING** :

* The panoramasdk wrapper class used in this demo is not the original Panorama sdk that is on the device image
* The wrapper class does not reflect the capabilities of the original Panorama SDK on the device
* Its sole purpose is to give a developer a realistic idea of the structure and signature of the sdk on the device

**Pre -Requisites**:
* Sagemaker Instance created with the right role (Policies needed IOT, Lambda and S3, IAM Full Access) 



**Frames to Process**:

* By default, we only process 10 frames from the video. If you want to increase this, please change this value in /panorama_sdk/panoramasdk.py and change frames_to_process = 10 to a value of your choice

#### Video to use

In [ ]:
import os
import sys
import pathlib

path = os.path.abspath(os.path.join(os.path.dirname("panorama_sdk"), '../..'))
sys.path.insert(1, path + '/panorama_sdk')

import jupyter_utils

In [ ]:
video_to_use = "TownCentreXVID.avi"

#### Let the Emulator know the following

1) That this is not an MXNet model zoo model  
2) That this is a custom Object Detection model  
3) That the framework this is built in is Tensorflow  
4) The task at hand is object detection  

In [ ]:
jupyter_utils.declare_globals({'mxnet_modelzoo_example': False, 
                               'custom_model': True, 'task':'object_detection', 'framework':'tensorflow'})

#### Import Tensorflow Dependencies

In [ ]:
!pip install tf_slim
!pip install pycocotools

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone -b "v2.3.0" https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash
cd models/research
pip install .

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import numpy as np
import os
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import tarfile
import shutil

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v2

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Enable Eager Execution for Tensorflow Version < 2
tf.compat.v1.enable_eager_execution()


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

## Looking at Individual Components of the code 

In [ ]:
import panoramasdk
import cv2
import numpy as np

def preprocess(img):
    resized = cv2.resize(img, (300, 300))
    x1 = np.asarray(resized)
    #np_img = np.expand_dims(x1, 0)
    return x1


print('Loading Model')
model = panoramasdk.model()
model.open('ssd_mobilenet_v1_coco_2017_11_17', 1)
print('Model Loaded')


class_info = model.get_output(0)
prob_info = model.get_output(3)
rect_info = model.get_output(2)

class_array = np.empty(class_info.get_dims(), dtype=class_info.get_type())
prob_array = np.empty(prob_info.get_dims(), dtype=prob_info.get_type())
rect_array = np.empty(rect_info.get_dims(), dtype=rect_info.get_type())

dogs_image = cv2.imread('lots_of_people.jpg')
x1 = preprocess(dogs_image)

model.batch(0, x1)
model.flush()

# To implement
resultBatchSet = model.get_result()

class_batch = resultBatchSet.get(0)
prob_batch = resultBatchSet.get(3)
rect_batch = resultBatchSet.get(2)

class_batch.get(0, class_array)
prob_batch.get(0, prob_array)
rect_batch.get(0, rect_array)

class_data = class_array[0]
prob_data = prob_array[0]
rect_data = rect_array[0]

## AWS Panorama Emulator for TF model

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output, Markdown, display
plt.rcParams["figure.figsize"] = (20,20)

jupyter_utils.change_video_source(video_to_use)

%matplotlib inline

In [ ]:
from __future__ import division
from __future__ import print_function
import panoramasdk
import cv2
import numpy as np
import boto3


class people_counter(panoramasdk.base):
    def interface(self):
        return {
            "parameters":
                (
                    ("float", "threshold", "Detection threshold", 0.4),
                    ("model", "people_counter", "Model for people counting", "ssd_mobilenet_v1_coco_2017_11_17"),
                    ("int", "batch_size", "Model batch size", 1),
                    ("float", "person_index", "person index based on dataset used", 1)
                ),
            "inputs":
                (
                    ("media[]", "video_in", "Camera input stream"),
                ),
            "outputs":
                (
                    ("media[video_in]", "video_out", "Camera output stream"),
                )
        }

    def init(self, parameters, inputs, outputs):
        try:
            # Detection probability threshold.
            self.threshold = parameters.threshold
            self.frame_num = 0
            # Number of People
            self.number_people = 0
            # Person Index for Model
            self.person_index = parameters.person_index
            # Set threshold for model
            self.threshold = parameters.threshold
            # Load model from the specified directory.
            print("loading the model...")
            self.model = panoramasdk.model()
            self.model.open(parameters.people_counter, 1)
            print("model loaded")
            # Create input and output arrays.
            class_info = self.model.get_output(0)
            prob_info = self.model.get_output(3)
            rect_info = self.model.get_output(2)
            self.class_array = np.empty(class_info.get_dims(), dtype=class_info.get_type())
            self.prob_array = np.empty(prob_info.get_dims(), dtype=prob_info.get_type())
            self.rect_array = np.empty(rect_info.get_dims(), dtype=rect_info.get_type())
            return True
        except Exception as e:
            print("Exception: {}".format(e))
            return False

    def preprocess(self, img):
        resized = cv2.resize(img, (300, 300))
        x1 = np.asarray(resized)
        #np_img = np.expand_dims(x1, 0)
        return x1

    def get_number_persons(self, class_data, prob_data):
        # get indices of people detections in class data
        person_indices = [i for i in range(len(class_data)) if class_data[i] == self.person_index]
        # use these indices to filter out anything that is less than 95% threshold from prob_data
        prob_person_indices = [i for i in person_indices if prob_data[i] >= self.threshold]
        return prob_person_indices

    def entry(self, inputs, outputs):
        for i in range(len(inputs.video_in)):
            stream = inputs.video_in[i]
            person_image = stream.image
            stream.add_label('Number : {}'.format(self.number_people), 0.8, 0.05)
            H, W, _ = person_image.shape
            x1 = self.preprocess(person_image)
            # Do inference on the new frame.
            self.model.batch(0, x1)
            self.model.flush()
            # Get the results.
            resultBatchSet = self.model.get_result()
            class_batch = resultBatchSet.get(0)
            prob_batch = resultBatchSet.get(3)
            rect_batch = resultBatchSet.get(2)
            
            class_batch.get(0, self.class_array)
            prob_batch.get(0, self.prob_array)
            rect_batch.get(0, self.rect_array)
            
            class_data = self.class_array[0]
            prob_data = self.prob_array[0]
            rect_data = self.rect_array[0]

            person_indices = self.get_number_persons(class_data, prob_data)
            self.number_people = len(person_indices)
            
            try:
                if self.number_people > 0:
                    for idx in person_indices:
                        top = rect_data[idx][0]
                        left = rect_data[idx][1]
                        bottom = rect_data[idx][2]
                        right = rect_data[idx][3]
                                            
                        stream.add_rect(left, top, right, bottom)
                        stream.add_label('Class, thresh : {}, {}'.format(class_data[idx], prob_data[idx]), left, top)


            except Exception as e:
                print('exception is {}'.format(e))
                

            stream.add_label('Number : {}'.format(self.number_people), 0.8, 0.05)
            self.model.release_result(resultBatchSet)
            
            outputs.video_out[i] = stream
            
        return True

In [ ]:
def main():
    """run() member function is implemented in the Lambda base class. It performs Lambda initialization and enters the main loop."""
    
    people_counter().run()
main()